# DFT

## Imports

In [1]:
import numpy as np
import scipy.linalg as spla
import pyscf
from pyscf import gto, scf, dft
import matplotlib.pyplot as plt
import time
%matplotlib notebook

## DFT
In HF, the expression for energy was written 
$$G_{\mu\nu} = \sum_{\lambda\sigma}^{\mathrm{num\_ao}} P_{\lambda \sigma}[2(\mu\nu|\lambda\sigma)-(\mu\lambda|\nu\sigma)]$$ 

$$G_{\mu\nu} = \sum_{\lambda\sigma}^{\mathrm{num\_ao}} P_{\lambda \sigma}[2J - K]$$ 

$$
E = \sum^{\mathrm{num\_ao}}_{\mu\nu} P_{\mu\nu} (H_{\mu\nu} + F_{\mu\nu})
$$

In DFT, the energy is written in terms of the electron density

$$E[n(r)] = T[n(r)] + V_{eN}[n(r)] + 0.5V_{coul}[n(r)] +E_{xc}[n(r)]$$

# STEP 1 : Specify the molecule

In [2]:
# start timer
start_time = time.time()
# define molecule
mol = pyscf.gto.M(
    atom="O 0.0000000 0.0000000 0.0000000; H 0.7569685 0.0000000 -0.5858752; H -0.7569685 0.0000000 -0.5858752",
    basis='sto-3g',
    unit="Ang",
    verbose=0,
    symmetry=False,
    spin=0,
    charge=0
)
# get number of atomic orbitals
num_ao = mol.nao_nr()
# get number of electrons
num_elec_alpha, num_elec_beta = mol.nelec
num_elec = num_elec_alpha + num_elec_beta
# get nuclear repulsion energy
E_nuc = mol.energy_nuc()

In [3]:
mol = pyscf.gto.M(
    atom="O 0.0000000 0.0000000 0.0000000; H 0.7569685 0.0000000 -0.5858752; H -0.7569685 0.0000000 -0.5858752",
    basis='sto-3g',
    unit="Ang",
    verbose=0,
    symmetry=False,
    spin=0,
    charge=0
)
mf = dft.RKS(mol)
mf.xc = 'lda'
print("LDA (PYSCF) = {}".format(mf.kernel()))

# get dft grid
grid_coords = mf.grids.coords
grid_weights = mf.grids.weights
ao_value = dft.numint.eval_ao(mol, grid_coords, deriv=0)

LDA (PYSCF) = -74.05975826788571


# STEP 2 : Calculate molecular integrals 

Overlap 

$$ S_{\mu\nu} = (\mu|\nu) = \int dr \phi^*_{\mu}(r) \phi_{\nu}(r) $$

Kinetic

$$ T_{\mu\nu} = (\mu\left|-\frac{\nabla}{2}\right|\nu) = \int dr \phi^*_{\mu}(r) \left(-\frac{\nabla}{2}\right) \phi_{\nu}(r) $$

Nuclear Attraction

$$ V_{\mu\nu} = (\mu|r^{-1}|\nu) = \int dr \phi^*_{\mu}(r) r^{-1} \phi_{\nu}(r) $$

Form Core Hamiltonian

$$ H = T + V $$


In [4]:
# calculate overlap integrals
S = mol.intor('cint1e_ovlp_sph')
# calculate kinetic energy integrals
T = mol.intor('cint1e_kin_sph')
# calculate nuclear attraction integrals
V = mol.intor('cint1e_nuc_sph')
# form core Hamiltonian
H = T + V
# since we are using the 8 fold symmetry of the 2 electron integrals
# the functions below will help us when accessing elements
__idx2_cache = {}

# calculate two electron integrals
eri = mol.intor('cint2e_sph', aosym='s8')


def idx2(i, j):
    if (i, j) in __idx2_cache:
        return __idx2_cache[i, j]
    elif i >= j:
        __idx2_cache[i, j] = int(i*(i+1)/2+j)
    else:
        __idx2_cache[i, j] = int(j*(j+1)/2+i)
    return __idx2_cache[i, j]


def idx4(i, j, k, l):
    return idx2(idx2(i, j), idx2(k, l))



# STEP 3 : Form guess density matrix

In [5]:
# set inital density matrix to zero
D = np.zeros((num_ao, num_ao))

DFT Algorithm

In [6]:
# 2 helper functions for printing during SCF
def print_start_iterations():
    print("{:^79}".format("{:>4}  {:>11}  {:>11}  {:>11}  {:>11}".format(
        "Iter", "Time(s)", "RMSC DM", "delta E", "E_elec")))
    print("{:^79}".format("{:>4}  {:>11}  {:>11}  {:>11}  {:>11}".format(
        "****", "*******", "*******", "*******", "******")))


def print_iteration(iteration_num, iteration_start_time, iteration_end_time, iteration_rmsc_dm, iteration_E_diff, E_elec):
    print("{:^79}".format("{:>4d}  {:>11f}  {:>.5E}  {:>.5E}  {:>11f}".format(iteration_num,
                                                                              iteration_end_time - iteration_start_time, iteration_rmsc_dm, iteration_E_diff, E_elec)))


# set stopping criteria
iteration_max = 100
convergence_E = 1e-9
convergence_DM = 1e-5
# loop variables
iteration_num = 0
E_total = 0
E_elec = 0.0
iteration_E_diff = 0.0
iteration_rmsc_dm = 0.0
converged = False
exceeded_iterations = False

In [7]:
def diis(F_list, diis_res):
    # Build B matrix
    dim_B = len(F_list) + 1
    B = np.empty((dim_B, dim_B))
    B[-1, :] = -1
    B[:, -1] = -1
    B[-1, -1] = 0
    for i in range(len(F_list)):
        for j in range(len(F_list)):
            B[i, j] = np.einsum('ij,ij->', diis_res[i], diis_res[j])

    # Right hand side of Pulay eqn
    right = np.zeros(dim_B)
    right[-1] = -1

    # Solve Pulay for coeffs
    cn = np.linalg.solve(B, right)

    # Build DIIS Fock
    F_diis = np.zeros_like(F_list[0])
    for x in range(cn.shape[0] - 1):
        F_diis += cn[x] * F_list[x]

    return F_diis




# Trial & Residual vector lists
F_list = []
DIIS_resid = []
# AO orthogonalization matrix
A = spla.fractional_matrix_power(S, -0.5)

print_start_iterations()
while (not converged and not exceeded_iterations):
    # store last iteration and increment counters
    iteration_start_time = time.time()
    iteration_num += 1
    E_elec_last = E_elec
    D_last = np.copy(D)
    # form G matrix
    G = np.zeros((num_ao, num_ao))
    exc = 0.0
    if iteration_num > 1:
        n, exc, vxc = mf._numint.nr_rks(mol, mf.grids, mf.xc, D)
        for i in range(num_ao):
            for j in range(num_ao):
                for k in range(num_ao):
                    for l in range(num_ao):
                        G[i, j] += D[k, l] * \
                            (2.0*(eri[idx4(i, j, k, l)])) 
        for i in range(num_ao):
            for j in range(num_ao):
                 G[i, j] += vxc[i,j]
    # build fock matrix
    F = H + G
    
    # =======> Start of DIIS stuff <=========
    # Build the DIIS AO gradient
    if iteration_num >=10:
        diis_r = A.T @ (F @ D @ S - S @ D @ F) @ A

        # DIIS RMS
        diis_rms = np.mean(diis_r**2)**0.5

        # Append lists
        F_list.append(F)
        DIIS_resid.append(diis_r)

        if iteration_num >= 15:
            # preform DIIS to get Fock Matrix
            F = diis(F_list, DIIS_resid)
            
            
    # solve the generalized eigenvalue problem
    E_orbitals, C = spla.eigh(F, S)
    # compute new density matrix
    D = np.zeros((num_ao, num_ao))
    for i in range(num_ao):
        for j in range(num_ao):
            for k in range(num_elec_alpha):
                D[i, j] += C[i, k] * C[j, k]
    # calculate electronic energy
    
    E_elec = np.sum(np.multiply(D, (H + F)))+ exc
    
    
    
    # calculate energy change of iteration
    iteration_E_diff = np.abs(E_elec - E_elec_last)
    # rms change of density matrix
    iteration_rmsc_dm = np.sqrt(np.sum((D - D_last)**2))
    iteration_end_time = time.time()
    print_iteration(iteration_num, iteration_start_time,
                    iteration_end_time, iteration_rmsc_dm, iteration_E_diff, E_elec)
    if(np.abs(iteration_E_diff) < convergence_E and iteration_rmsc_dm < convergence_DM):
        converged = True
    if(iteration_num == iteration_max):
        exceeded_iterations = True

           Iter      Time(s)      RMSC DM      delta E       E_elec            
           ****      *******      *******      *******       ******            
              1     0.000553  2.69561E+00  1.27367E+02  -127.366748            
              2     0.018192  2.34183E+00  5.03077E+01   -77.059094            
              3     0.016119  2.04317E+00  1.01318E+01   -87.190936            
              4     0.015745  1.75887E+00  8.49074E+00   -78.700191            
              5     0.016424  1.67237E+00  7.67842E+00   -86.378611            
              6     0.021860  1.56852E+00  7.21961E+00   -79.159003            
              7     0.012774  1.52798E+00  6.89795E+00   -86.056949            
              8     0.018775  1.47768E+00  6.68479E+00   -79.372159            
              9     0.028515  1.45587E+00  6.52288E+00   -85.895037            
             10     0.013411  1.42848E+00  6.40861E+00   -79.486423            
             11     0.021489  1.41596E+0

# STEP 9 : Calculate Observables

In [8]:
# calculate total energy
E_total = E_elec + E_nuc


In [9]:
print("{:^79}".format("Total Energy : {:>11f}".format(E_total)))

                          Total Energy :  -73.452150                           


In [10]:
mol = pyscf.gto.M(
    atom="O 0.0000000 0.0000000 0.0000000; H 0.7569685 0.0000000 -0.5858752; H -0.7569685 0.0000000 -0.5858752",
    basis='sto-3g',
    unit="Ang",
    verbose=0,
    symmetry=False,
    spin=0,
    charge=0
)
mf = dft.RKS(mol)
mf.xc = 'lda'
print("LDA (PYSCF) = {}".format(mf.kernel()))

LDA (PYSCF) = -74.05975826788568
